In [1]:
# Import packages
import os
import geopandas as gpd
import numpy as np
from osgeo import gdal
import rasterio
import rasterio.mask
import netCDF4 as nc

In [2]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\geospatial_internship\\datasets')
shape_telangana = gpd.read_file('telangana_shapefile.geojson')
shape_telangana = shape_telangana.explode()

C:\Users\Jesse\AppData\Local\Temp\ipykernel_14040\1639596723.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  shape_telangana = shape_telangana.explode()


In [ ]:
folder_names = os.listdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\PM2.5\\')
for i in folder_names:
    file_names = os.listdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\PM2.5\\' + i)
    for j in file_names:
        nc_data = "C:\\Users\\Jesse\\OneDrive\\Documenten\\PM2.5\\" + i + "\\" + j
        nc_file = gdal.Open('NETCDF:'+ nc_data + ':GWRPM25')
        # Convert the netCDF to Geotiff file
        tiff_file = gdal.Translate('C:\\Users\\Jesse\\OneDrive\\Documenten\\Tiff\\'+ j + '.tif', nc_file)

In [3]:
path = 'C:\\Users\\Jesse\\OneDrive\\Documenten\\PM2.5\\2015\\V5GL02.HybridPM25.Asia.201501-201501.nc'

In [5]:
nc_metadata = nc.Dataset(path)
nc_metadata['GWRPM25']

<class 'netCDF4._netCDF4.Variable'>
float32 GWRPM25(lat, lon)
    long_name: Hybrid PM_2_._5 [\mug/m^3]
    units: [\mug/m^3]
unlimited dimensions: 
current shape = (5500, 8000)
filling on, default _FillValue of 9.969209968386869e+36 used

In [11]:
# Import the netCDF data for Leaf Area Index
nc_file = gdal.Open('NETCDF:c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\c_gls_BA300_202205310000_GLOBE_S3_V1.2.1.nc:NOBS')
# Convert the netCDF to Geotiff file
tif_file = gdal.Translate('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_tiff.tif', nc_file)

In [13]:
# In the next two code blocks, we crop the Geotiff file to Burnt Area in Telangana
ba_data = "c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_tiff.tif"
with rasterio.open(ba_data) as src:
    out_image, out_transform = rasterio.mask.mask(src, shape_telangana.geometry, crop=True)
    out_meta = src.meta

In [14]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# Save the Geotiff file to Burnt Area in Telangana
with rasterio.open("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_cropped_tiff.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    

In [3]:
# For each of the BA images, we crop the Geotiff file to BA in Telangana
folder_names = os.listdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Input')
for i in folder_names:
    file_names = os.listdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Input\\' + i)
    for j in file_names:
        BA_data = "c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Input\\" + i + "/" + j
        with rasterio.open(BA_data) as src:
            out_image, out_transform = rasterio.mask.mask(src, shape_telangana.geometry, crop=True)
            out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        # photometric = 'YCbCr'
        # photometric = 'RGB', compress = 'JPEG'
        # Save the Geotiff file to BA in Telangana
        with rasterio.open("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\Output\\" + j, "w", **out_meta) as dest:
            dest.write(out_image)

In [ ]:
#for %N in (C:/Users/Jesse/OneDrive/Documenten/Output/*.tiff) DO gdal_translate C:/Users/Jesse/OneDrive/Documenten/Output/%~nN.tiff  C:/Users/Jesse/OneDrive/Documenten/COG/%~nN.tiff -co COMPRESS=LZW -co TILED=YES